`April 21, 2022`

## Collaborative Filtering: Model Based

> ## Library

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 3.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618250 sha256=146da6b286bacfde843e2c004ab24c9db238be5deae0e583f3c146fabc244d78
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import pandas as pd

import seaborn as sns

from surprise import Reader
from surprise import Dataset

from surprise import SVD
from surprise import BaselineOnly

from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.model_selection import GridSearchCV

> ## Data

In [ ]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

In [ ]:
df

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
...,...,...,...,...
99998,880,476,3,880175444
99999,716,204,5,879795543
100000,276,1090,1,874795795
100001,13,225,2,882399156


In [ ]:
user_item_rating_matrix = df.pivot_table(values = 'rating', index = 'user_id', columns = 'item_id')

In [ ]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id','item_id','rating']], reader)

In [ ]:
data.df.head(10)

,user_id,item_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3
5,22,377,1
6,244,51,2
7,166,346,1
8,298,474,4
9,115,265,2


> ## Validation

Data Splitting

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25,random_state=101)

Algo 1 : SVD

In [ ]:
algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9398


0.9397981717733036

Algo 2 : ALS

In [ ]:
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...


In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9409


0.9408558035131163

Based on train-test validation, SVD is better than ALS

> ## Cross Validation

Algo 1 : SVD

In [ ]:
algo = SVD()

cv_svd = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9393  0.9382  0.9425  0.9332  0.9293  0.9365  0.0047  
MAE (testset)     0.7376  0.7407  0.7399  0.7361  0.7352  0.7379  0.0021  
Fit time          4.95    4.89    4.89    4.84    4.88    4.89    0.04    
Test time         0.14    0.23    0.14    0.24    0.16    0.18    0.04    


In [ ]:
print('rmse cv mean',cv_svd['test_rmse'].mean())

rmse cv mean 0.9365023535802692


Algo 2 : ALS

In [ ]:
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }

algo = BaselineOnly(bsl_options=bsl_options)

cv_als = cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


In [ ]:
print('rmse cv mean',cv_als['test_rmse'].mean())

rmse cv mean 0.9415166339469463


Based on cross validation, SVD is better than ALS

> ## Hyperparameter Tuning

We optimize SVD further using hyperparameter tuning

In [ ]:
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005],
              'reg_all': [0.02, 0.4, 0.6]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

best score and best hyperparameter

In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.946206662876543
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [ ]:
print(gs.best_score['mae'])
print(gs.best_params['mae'])

0.7470844480823832
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


before

In [ ]:
algo = SVD()

cv_svd = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9345  0.9396  0.9323  0.9350  0.9444  0.9372  0.0043  
MAE (testset)     0.7371  0.7390  0.7345  0.7380  0.7459  0.7389  0.0038  
Fit time          4.82    4.82    4.82    4.82    4.81    4.82    0.00    
Test time         0.14    0.14    0.24    0.14    0.14    0.16    0.04    


In [ ]:
print('rmse cv mean',cv_svd['test_rmse'].mean())

rmse cv mean 0.9371529626957331


after

In [ ]:
algo = SVD(n_epochs = 20, lr_all = 0.005, reg_all = 0.02)
cv_svd = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9402  0.9373  0.9302  0.9334  0.9276  0.9337  0.0046  
MAE (testset)     0.7407  0.7373  0.7362  0.7356  0.7309  0.7361  0.0032  
Fit time          4.86    4.89    4.95    4.92    4.91    4.91    0.03    
Test time         0.26    0.14    0.26    0.14    0.14    0.19    0.06    


In [ ]:
print('rmse cv mean',cv_svd['test_rmse'].mean())

rmse cv mean 0.9337282090558603


> ## Prediction Result

We will reccommend item 565, 647, 665, and 677 to users 0, 111 and 212.

In [ ]:
df_test = pd.DataFrame(columns = ['user_id','item_id'])

for i in [0,111,212]:
    for j in [647, 665, 565, 677]:
        df_test = df_test.append({'user_id':i,'item_id':j},ignore_index = True)
        
df_test

,user_id,item_id
0,0,647
1,0,665
2,0,565
3,0,677
4,111,647
5,111,665
6,111,565
7,111,677
8,212,647
9,212,665


Build the final model using SVD with following hyperparameter, n_epochs = {5, 10}, lr_all = {0.002, 0.005}, reg_all = {0.4, 0.6}

In [ ]:
algo = SVD(n_epochs = 20, lr_all = 0.005, reg_all = 0.02)
algo.fit(trainset)

y = []

for _, row in df_test.iterrows():
    est = algo.predict(row.user_id, row.item_id)
    y.append(est[3])
    
df_test['rating'] = y

df_test.sort_values(by = ['user_id','rating'], ascending = [True,False], inplace = True)

USER ID - 0

In [ ]:
df_test[df_test['user_id'] == 0]

,user_id,item_id,rating
0,0,647,3.941630
3,0,677,3.310805
2,0,565,3.085720
1,0,665,3.040065


recommendation order for user 0 : 647, 677, 665, 565

USER ID - 111

In [ ]:
df_test[df_test['user_id'] == 111]

,user_id,item_id,rating
4,111,647,3.903866
7,111,677,3.249486
5,111,665,3.002651
6,111,565,2.941813


recommendation order for user 0 : 647, 677, 565, 665

USER ID - 212

In [ ]:
df_test[df_test['user_id'] == 212]

,user_id,item_id,rating
8,212,647,3.844641
11,212,677,3.419403
9,212,665,2.886898
10,212,565,2.863581


recommendation order for user 0 : 647, 677, 565, 665